In [1]:
#!pip install cudaq

In [1]:
import cudaq
import sys
import numpy as np
from math import floor
import auxiliary_files.labs_utils as utils
import time
try:
    import cupy as cp
    DEVICE = "GPU (CuPy)"
except ImportError:
    import numpy as cp
    DEVICE = "CPU (NumPy)"

try:
    cudaq.set_target("nvidia")
    print("Successfully set target to NVIDIA GPU.")
except:
    print("NVIDIA GPU target not found. Falling back to CPU.")

Successfully set target to NVIDIA GPU.


In [2]:
N = 20

In [3]:
def calculate_energy_fft(sequences):
    """
    LABS probleminin enerji fonksiyonu: E = sum_{k=1}^{N-1} (C_k)^2
    Wiener-Khinchin teoremi kullanılarak O(N log N) hızında hesaplanır.
    """
    if sequences.ndim == 1:
        sequences = sequences.reshape(1, -1)

    batch_size, N = sequences.shape
    # FFT hızı için en yakın 2'nin kuvvetine tamamla
    n_fft = 2**((2 * N - 1).bit_length())

    # Fourier Dönüşümü ile otokorelasyon hesabı
    f_seq = cp.fft.rfft(sequences, n=n_fft, axis=1)
    autocorr = cp.fft.irfft(f_seq * cp.conj(f_seq), n=n_fft, axis=1)

    # k=1'den N-1'e kadar olan kısımların kareleri toplamı (C_k^2)
    # cp.round kullanıyoruz çünkü FFT float sonuç döner, LABS tam sayıdır.
    energies = cp.sum(cp.round(autocorr[:, 1:N])**2, axis=1)
    return energies

# --- [STEP 3: PARALEL ADAY LİSTELİ TABU SEARCH (EDUCATION)] ---
def tabu_search_gpu(initial_seq, max_iters=30, tabu_tenure=7):
    """
    Yerel arama adımı. GPU'da tüm komşuları aynı anda değerlendirir.
    """
    N = len(initial_seq)
    current_seq = cp.array(initial_seq)
    current_energy = calculate_energy_fft(current_seq)[0]

    best_seq = current_seq.copy()
    best_energy = current_energy
    tabu_list = cp.zeros(N, dtype=cp.int32)

    for it in range(max_iters):
        # Tüm 1-bit flip komşuları tek bir matriste oluştur (N, N)
        neighbors = cp.tile(current_seq, (N, 1))
        diag_indices = cp.arange(N)
        neighbors[diag_indices, diag_indices] *= -1

        # Enerjileri toplu halde (batch) hesapla
        neighbor_energies = calculate_energy_fft(neighbors)

        # Tabu durumu ve Aspiration (En iyiyi geçiyorsa yasağı del)
        is_not_tabu = (tabu_list <= it)
        is_aspiration = (neighbor_energies < best_energy)
        mask = is_not_tabu | is_aspiration

        neighbor_energies[~mask] = cp.inf

        # En iyi hareketi seç
        best_move_idx = cp.argmin(neighbor_energies)
        current_seq = neighbors[best_move_idx]
        current_energy = neighbor_energies[best_move_idx]

        if current_energy < best_energy:
            best_energy = current_energy
            best_seq = current_seq.copy()

        # Hamleyi yasaklılar listesine ekle
        tabu_list[best_move_idx] = it + tabu_tenure

    return best_seq, float(best_energy)

# --- [STEP 4: ANA MEMETİK TABU SEARCH DÖNGÜSÜ] ---
def run_full_mts(N=64, pop_size=20, generations=50, p_mutate=0.2, initial_population = None):
    """
    Memetik algoritmanın ana döngüsü: Combine -> Mutate -> Educate
    """
    # Popülasyonu başlat (İleride buraya Kuantum Seeds eklenebilir)
    if initial_population is not None:
            # Arkadaşından gelen listeyi GPU matrisine çevir
            quantum_pop = cp.array(initial_population)

            # Eğer kuantumdan gelen dizi sayısı pop_size'dan küçükse geri kalanı rastgele doldur
            if len(quantum_pop) < pop_size:
                extra_count = pop_size - len(quantum_pop)
                extra_pop = cp.random.choice(cp.array([-1, 1]), (extra_count, N))
                population = cp.vstack([quantum_pop, extra_pop])
            else:
                # Eğer kuantumdan çok fazla dizi gelirse en iyilerini/ilklerini al
                population = quantum_pop[:pop_size]
    else:
            # Dışarıdan veri gelmezse tamamen rastgele başlat
        population = cp.random.choice(cp.array([-1, 1]), (pop_size, N))

    global_best_seq = None
    global_best_energy = float('inf')
    history = []

    start_time = time.time()

    for gen in range(generations):
        # 1. COMBINE (Crossover): 3 ebeveynden çoğunluk oyu
        idx = cp.random.choice(pop_size, 3, replace=False)
        child = cp.sign(cp.sum(population[idx], axis=0))
        child[child == 0] = 1

        # 2. MUTATE: Rastgele bit çevirme
        if cp.random.random() < p_mutate:
            m_point = cp.random.randint(0, N)
            child[m_point] *= -1

        # 3. EDUCATION: Tabu Search ile yerel iyileştirme
        improved_child, improved_energy = tabu_search_gpu(child)

        # 4. SELECTION: Popülasyondaki en kötünün yerine koy (Steady-state update)
        pop_energies = calculate_energy_fft(population)
        worst_idx = cp.argmax(pop_energies)

        if improved_energy < pop_energies[worst_idx]:
            population[worst_idx] = improved_child

        # Global Best Güncelleme
        if improved_energy < global_best_energy:
            global_best_energy = improved_energy
            global_best_seq = improved_child.copy()

        history.append(global_best_energy)
        if gen % 10 == 0:
            print(f"Gen {gen:03d} | En İyi Enerji: {global_best_energy:.1f}")

    total_duration = time.time() - start_time
    return global_best_seq, global_best_energy, total_duration, history
best_sol, best_en, duration, hist = run_full_mts(N=N, generations=100)

Gen 000 | En İyi Enerji: 50.0
Gen 010 | En İyi Enerji: 34.0
Gen 020 | En İyi Enerji: 34.0
Gen 030 | En İyi Enerji: 26.0
Gen 040 | En İyi Enerji: 26.0
Gen 050 | En İyi Enerji: 26.0
Gen 060 | En İyi Enerji: 26.0
Gen 070 | En İyi Enerji: 26.0
Gen 080 | En İyi Enerji: 26.0
Gen 090 | En İyi Enerji: 26.0


In [4]:
# TODO  Write CUDA-Q kernels to apply the 2 and 4 qubit operators.
@cudaq.kernel
def r_zz(theta: float, q0: cudaq.qubit, q1: cudaq.qubit):
    """
    Helper kernel for exp(-i * theta/2 * Z * Z).
    Standard decomposition: CNOT -> Rz -> CNOT
    """
    x.ctrl(q0, q1)
    rz(theta, q1)
    x.ctrl(q0, q1)

@cudaq.kernel
def r_yz(theta: float, q_y: cudaq.qubit, q_z: cudaq.qubit):
    """
    Applies the 2-body interaction: exp(-i * theta/2 * Y * Z).
    """
    # 1. Rotate Y qubit to Z basis (Rx(pi/2))
    rx(np.pi / 2, q_y)

    # 2. Apply standard ZZ interaction
    r_zz(theta, q_y, q_z)

    # 3. Rotate back (Rx(-pi/2))
    rx(-np.pi / 2, q_y)

@cudaq.kernel
def r_zy(theta: float, q_z: cudaq.qubit, q_y: cudaq.qubit):
    """
    Applies the 2-body interaction: exp(-i * theta/2 * Z * Y).
    Symmetric to r_yz, just swapping the basis change target.
    """
    rx(np.pi / 2, q_y)
    r_zz(theta, q_z, q_y)
    rx(-np.pi / 2, q_y)

@cudaq.kernel
def r_4body_yzzz(theta: float, q_y: cudaq.qubit, q_z1: cudaq.qubit, q_z2: cudaq.qubit, q_z3: cudaq.qubit):
    """
    Applies the 4-body interaction: exp(-i * theta/2 * Y * Z * Z * Z).

    Note: This kernel can be used for any variation (ZYZZ, ZZYZ, ZZZY)
    by simply passing the qubit intended for the 'Y' term as the first argument (q_y).
    """
    # 1. Rotate Y qubit to Z basis
    rx(np.pi / 2, q_y)

    # 2. CNOT Ladder to compute parity onto the last qubit (q_z3)
    x.ctrl(q_y, q_z1)
    x.ctrl(q_z1, q_z2)
    x.ctrl(q_z2, q_z3)

    # 3. Apply Phase Rotation
    rz(theta, q_z3)

    # 4. Uncompute Parity (Reverse Ladder)
    x.ctrl(q_z2, q_z3)
    x.ctrl(q_z1, q_z2)
    x.ctrl(q_y, q_z1)

    # 5. Restore Basis
    rx(-np.pi / 2, q_y)

In [5]:
def get_interactions(N):
    """
    Generates the interaction sets G2 and G4 based on the loop limits in Eq. 15.
    Returns standard 0-based indices as lists of lists of ints.

    Args:
        N (int): Sequence length.

    Returns:
        G2: List of lists containing two body term indices [i, i+k]
        G4: List of lists containing four body term indices [i, i+t, i+k, i+k+t]
    """
    G2 = []
    G4 = []

    # 1. Generate G2 (2-Body Interactions)
    # Formula: Product_{i=1}^{N-2} Product_{k=1}^{floor((N-i)/2)}
    for i in range(N - 2):  # i is 0-based here, corresponds to math 'i-1'
        # In formula, limit is floor((N - i_math) / 2)
        # i_math = i + 1
        k_limit = floor((N - (i + 1)) / 2)

        for k in range(1, k_limit + 1):
            # Indices are i and i+k
            G2.append([i, i + k])

    # 2. Generate G4 (4-Body Interactions)
    # Formula: Product_{i=1}^{N-3} Product_{t=1}^{floor((N-i-1)/2)} Product_{k=t+1}^{N-i-t}
    for i in range(N - 3):
        # i_math = i + 1
        t_limit = floor((N - (i + 1) - 1) / 2)

        for t in range(1, t_limit + 1):
            k_start = t + 1
            k_end = N - (i + 1) - t  # Inclusive in math, so +1 for range

            for k in range(k_start, k_end + 1):
                # Indices: i, i+t, i+k, i+k+t
                # Note: These correspond to the positions of the Z/Y operators
                G4.append([i, i + t, i + k, i + k + t])

    return G2, G4

In [6]:
@cudaq.kernel
def trotterized_circuit(N: int, G2: list[list[int]], G4: list[list[int]],
                        steps: int, dt: float, T: float, thetas: list[float],
                        biases: list[float]): # <--- Added biases input

    # 1. Allocate Qubits and Initialize
    q = cudaq.qvector(N)
    h(q) # Start in |+> state

    # 2. Trotter Loop
    for step in range(steps):

        # Get the theta for this specific time step
        theta = thetas[step]

        # Calculate base angles based on paper Eq 15
        angle_2body = 4.0 * theta
        angle_4body = 8.0 * theta

        # --- A. Apply Bias Fields ---
        # REMOVED: if abs(biases[i]) > 1e-5 check.
        # Just apply the rotation directly. If biases[i] is 0, this is an Identity gate.
        for i in range(N):
            rz(2.0 * biases[i] * dt, q[i])

        # --- B. Apply 2-Body Terms ---
        for pair in G2:
            i = pair[0]
            k = pair[1]
            r_yz(angle_2body, q[i], q[k])
            r_zy(angle_2body, q[i], q[k])

        # --- C. Apply 4-Body Terms ---
        for quad in G4:
            idx0 = quad[0]
            idx1 = quad[1]
            idx2 = quad[2]
            idx3 = quad[3]

            r_4body_yzzz(angle_4body, q[idx0], q[idx1], q[idx2], q[idx3])
            r_4body_yzzz(angle_4body, q[idx1], q[idx0], q[idx2], q[idx3])
            r_4body_yzzz(angle_4body, q[idx2], q[idx0], q[idx1], q[idx3])
            r_4body_yzzz(angle_4body, q[idx3], q[idx0], q[idx1], q[idx2])

    # 3. Measurement
    mz(q)

In [7]:
T = 1.0              # Annealing time (Paper often suggests T=1.0 - 2.0 for BF-DCQO)
n_steps = 10          # Number of Trotter steps
bf_iterations = 5    # Number of Bias-Field iterations (Ref: 3-5 iterations are common)
dt = T / n_steps

# 1. Prepare Interactions & Schedule
G2, G4 = get_interactions(N)

# Compute theta schedule
thetas = []
for step in range(1, n_steps + 1):
    t = step * dt
    # Ensure utils.compute_theta is available or define the schedule explicitly
    # If utils is missing, use a linear schedule for demonstration: theta_val = t/T * (1-t/T)
    theta_val = utils.compute_theta(t, dt, T, N, G2, G4)
    thetas.append(theta_val)

print(f"--- Phase 1: Bias-Field DCQO (BF-DCQO) N={N} ---")
print(f"Iterations: {bf_iterations}, Trotter Steps: {n_steps}")

# Initialize Biases to 0.0
current_biases = [0.0] * N

# 2. Iterative BF-DCQO Loop
for it in range(bf_iterations):
    print(f"  > Iteration {it+1}/{bf_iterations}...")

    # A. Run Quantum Circuit with current biases
    # Note: We sample enough shots to estimate expectation values <Z_i>
    results = cudaq.sample(trotterized_circuit, N, G2, G4, n_steps, dt, T, thetas, current_biases, shots_count=2000)

    # B. Compute Expectation Values <Z_i> from shots
    # Iterate over measured bitstrings to calculate magnetization
    magnetization = np.zeros(N)
    total_counts = 0

    for bitstring, count in results.items():
        total_counts += count
        # Convert string to array: '0' -> +1, '1' -> -1
        spins = np.array([1 if b == '0' else -1 for b in bitstring])
        magnetization += spins * count

    avg_magnetization = magnetization / total_counts

    # C. Update Biases for next iteration
    # Formula: h_i(new) = h_i(old) + alpha * <Z_i>
    # The paper suggests accumulating biases to "freeze" stable qubits
    alpha = 0.5  # Update rate / Feedback strength
    for i in range(N):
        current_biases[i] += alpha * avg_magnetization[i]

    print(f"    Max Bias: {np.max(np.abs(current_biases)):.3f}, Max <Z>: {np.max(np.abs(avg_magnetization)):.3f}")

# 3. Extract Elite Seeds from the FINAL iteration
print("  > Extracting Seeds from final distribution...")
# We take the top bitstrings from the final biased distribution
top_candidates = sorted(results.items(), key=lambda x: x[1], reverse=True)[:20]

quantum_seeds = []
for bitstring, count in top_candidates:
    spin_seq = [1 if b == '0' else -1 for b in bitstring]
    quantum_seeds.append(spin_seq)

print(f"Generated {len(quantum_seeds)} high-quality seeds using BF-DCQO.")
print(f"Top Candidate Count: {top_candidates[0][1]} shots")

# 5. RUN HYBRID MEMETIC TABU SEARCH
print(f"\n--- Phase 2: Classical Optimization (Hybrid) ---")
print("Starting MTS with Quantum-Seeded Population...")

# Run MTS using the quantum seeds
best_seq, best_energy, final_pop, history = run_full_mts(
    N=N,
    pop_size=len(quantum_seeds),   # Match population size to our seeds
    generations=50,
    initial_population=quantum_seeds # <--- INJECT QUANTUM DATA HERE
)
# 6. REPORT RESULTS
print(f"Total annealing time {T}")
print(f"Number of Trotter steps: {n_steps}")
print("\n--- Final Results ---")
print(f"Best Energy Found: {best_energy}")
print(f"Best Sequence: {best_seq}")

print(f"Random Start Energy: {best_en}")
print(f"Random Best Sequence: {best_sol}")
print(f"Quantum Boost: {best_en - best_energy} energy reduction")

--- Phase 1: Bias-Field DCQO (BF-DCQO) N=20 ---
Iterations: 5, Trotter Steps: 10
  > Iteration 1/5...
    Max Bias: 0.019, Max <Z>: 0.039
  > Iteration 2/5...
    Max Bias: 0.023, Max <Z>: 0.050
  > Iteration 3/5...
    Max Bias: 0.026, Max <Z>: 0.033
  > Iteration 4/5...
    Max Bias: 0.045, Max <Z>: 0.063
  > Iteration 5/5...
    Max Bias: 0.050, Max <Z>: 0.047
  > Extracting Seeds from final distribution...
Generated 20 high-quality seeds using BF-DCQO.
Top Candidate Count: 2 shots

--- Phase 2: Classical Optimization (Hybrid) ---
Starting MTS with Quantum-Seeded Population...
Gen 000 | En İyi Enerji: 34.0
Gen 010 | En İyi Enerji: 34.0
Gen 020 | En İyi Enerji: 34.0
Gen 030 | En İyi Enerji: 26.0
Gen 040 | En İyi Enerji: 26.0
Total annealing time 1.0
Number of Trotter steps: 10

--- Final Results ---
Best Energy Found: 26.0
Best Sequence: [ 1 -1 -1  1  1  1 -1 -1  1 -1  1  1  1 -1  1 -1 -1 -1 -1 -1]
Random Start Energy: 26.0
Random Best Sequence: [ 1  1 -1 -1  1 -1 -1  1  1  1  1 -1  

In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt

# --- BENCHMARK CONFIGURATION ---
N_MIN = 25
N_MAX = 34  # Milestone 3 hedefi
bench_results = []

print(f"Starting Benchmark: N={N_MIN} to N={N_MAX}")
print("-" * 50)

for current_n in range(N_MIN, N_MAX + 1,2):
    print(f"Processing N={current_n}...", end=" ")
    
    # 1. CLASSICAL ONLY RUN (Baseline)
    start_time_classic = time.time()
    _, best_en_classic, _, _ = run_full_mts(
    N=current_n,
    pop_size=20,   # Match population size to our seeds
    generations=50,
)
    end_time_classic = time.time()
    classic_duration = end_time_classic - start_time_classic

    # 2. QUANTUM-HYBRID RUN
    start_time_hybrid = time.time()
    
    # Step 2a: Quantum Seeding
    G2, G4 = get_interactions(current_n)
    thetas = [utils.compute_theta(t*0.04, 0.04, 0.2, current_n, G2, G4) for t in range(1, 6)]
    results = cudaq.sample(trotterized_circuit, current_n, G2, G4, 5, 0.04, 0.2, thetas,current_biases, shots_count=500)

    # En iyi 10 bitstring'i tohum olarak al
    top_candidates = sorted(results.items(), key=lambda x: x[1], reverse=True)[:10]
    q_seeds = [[1 if b == '0' else -1 for b in bitstring] for bitstring, _ in top_candidates]
    
    # Step 2b: Hybrid MTS
    _, best_en_hybrid, _, _ = run_full_mts(N=current_n, initial_population=q_seeds, generations=50)
    
    end_time_hybrid = time.time()
    hybrid_duration = end_time_hybrid - start_time_hybrid
    
    # Store results
    bench_results.append({
        'N': current_n,
        'Classic_Energy': best_en_classic,
        'Hybrid_Energy': best_en_hybrid,
        'Classic_Time': classic_duration,
        'Hybrid_Time': hybrid_duration,
        'Speedup': classic_duration / hybrid_duration if hybrid_duration > 0 else 0
    })
    print(f"Done. (Classic: {best_en_classic}, Hybrid: {best_en_hybrid})")

# --- DATA ANALYSIS & PLOTTING ---
df = pd.DataFrame(bench_results)

# Grafik 1: Time Comparison (Computational Complexity)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(df['N'], df['Classic_Time'], 'r-o', label='Classical Only')
plt.plot(df['N'], df['Hybrid_Time'], 'b-s', label='Quantum Hybrid')
plt.title('Execution Time Comparison')
plt.xlabel('Sequence Length (N)')
plt.ylabel('Time (seconds)')
plt.yscale('log') # Zaman farkını daha net görmek için logaritmik eksen
plt.grid(True, which="both", ls="-", alpha=0.5)
plt.legend()

# Grafik 2: Quality Comparison (Energy Minima)
plt.subplot(1, 2, 2)
plt.plot(df['N'], df['Classic_Energy'], 'r--o', label='Classic Energy')
plt.plot(df['N'], df['Hybrid_Energy'], 'b--s', label='Hybrid Energy')
plt.title('Best Energy Found (Lower is Better)')
plt.xlabel('Sequence Length (N)')
plt.ylabel('Energy')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.savefig('complexity_benchmark.png')
plt.show()

# --- SUMMARY REPORT ---
print("\n--- Benchmark Summary Table ---")
print(df[['N', 'Classic_Energy', 'Hybrid_Energy', 'Classic_Time', 'Hybrid_Time']].to_string(index=False))

Starting Benchmark: N=25 to N=34
--------------------------------------------------
Processing N=25... Gen 000 | En İyi Enerji: 72.0
Gen 010 | En İyi Enerji: 68.0
Gen 020 | En İyi Enerji: 48.0
Gen 030 | En İyi Enerji: 48.0
Gen 040 | En İyi Enerji: 48.0
Gen 000 | En İyi Enerji: 64.0
Gen 010 | En İyi Enerji: 44.0
Gen 020 | En İyi Enerji: 44.0
Gen 030 | En İyi Enerji: 44.0
Gen 040 | En İyi Enerji: 44.0
Done. (Classic: 48.0, Hybrid: 44.0)
Processing N=27... Gen 000 | En İyi Enerji: 77.0
Gen 010 | En İyi Enerji: 69.0
Gen 020 | En İyi Enerji: 65.0
Gen 030 | En İyi Enerji: 65.0
Gen 040 | En İyi Enerji: 61.0
Gen 000 | En İyi Enerji: 89.0
Gen 010 | En İyi Enerji: 53.0
Gen 020 | En İyi Enerji: 53.0
Gen 030 | En İyi Enerji: 53.0
Gen 040 | En İyi Enerji: 53.0
Done. (Classic: 61.0, Hybrid: 53.0)
Processing N=29... Gen 000 | En İyi Enerji: 102.0
Gen 010 | En İyi Enerji: 78.0
Gen 020 | En İyi Enerji: 70.0
Gen 030 | En İyi Enerji: 70.0
Gen 040 | En İyi Enerji: 70.0
Gen 000 | En İyi Enerji: 94.0
Gen 01